In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML, display

In [9]:
#Get titles from one page
def titles(section, tag, attr, title_list):
    for i, h in enumerate(section.findAll(tag, attrs = attr)):
        link = h.findAll('span', {'class': None})
        if link:
            for a in link:
                for title in a:
                    title_list.append(title)

#Get dates from one page
def dates(section, tag, attr, date_list):
    for i, h in enumerate(section.findAll(tag, attrs = attr)):
        link = h.findAll('span', {'class': None})
        if link:
            for a in link:
                for art_date in a:
                    if '.20' in art_date:
                        date_list.append(art_date)

#Extracts article's link
def links(section, tag, link_list):
    for i, h in enumerate(section.findAll(tag)):
        #print(h['href'])
        link_list.append(h['href'])

#Collect titles and dates from one page
def collector(soup, tag, attr, title_list, date_list, link_list):
    for i, h in enumerate(soup.findAll(tag, attrs = attr)):
        article = h.findAll('div', {'class':'b-list__item'})
        if article:
            for section in article:
                titles(section, 'span', {'class':'b-list__item-title'}, title_list)
                dates(section, 'div', {'class':'b-list__item-date'}, date_list)
                links(section, 'a', link_list)

#Find 'show more' button on the page and extract it's redirect link
def find_button(soup, tag, attr):#'div', {'class':'b-pager'}
    button = ''
    for i, h in enumerate(soup.findAll(tag, attrs = attr)):
        link = h.findAll('a')
        if link:
            for a in link:
                button = a['data-ajax']
    return button

#html-parcer
def soup_fun(section):
    link = 'https://ria.ru/'
    req = requests.get(link + section)
    soup = BeautifulSoup(req.text, 'lxml')
    return soup

#walks through pages until meet the condition
def recursor(title_list, date_list, link_list, soup):
    print(date_list[-1])
    try:
        new_soup = soup_fun(find_button(soup, 'div', {'class':'b-pager'}))
        collector(new_soup, 'div', {'class':'b-list'}, title_list, date_list, link_list)
        if date_list[-1] != '01.01.2017' and int(date_list[-1][-4:]) >= 2017:
            recursor(title_list, date_list, link_list, new_soup)
    except TimeoutError:
        raise('Oooops, TimeoutError')

In [10]:
req_list = ['politics', 'society', 'economy', 'incidents', 'world'] 
title_list = []
date_list = []
link_list = []

for section in req_list:
    soup = soup_fun(section)
    collector(soup, 'div', {'class':'b-list'}, title_list, date_list, link_list)
    recursor(title_list, date_list, link_list, soup)
    print('title_list', len(title_list))
    print('date_list', len(date_list))
    print('link_list', len(link_list))
    print(title_list[-1])

    with open('titles_'+sect+'_till_01.01.2017.txt', 'w', encoding = 'utf-8') as f:
    #with open('titles_world_till_01.06.2017-01.01.2017.txt', 'w', encoding = 'utf-8') as f:
        for title in title_list:
            f.write(title + '\n')

26.05.2018
26.05.2018
25.05.2018
25.05.2018
24.05.2018
24.05.2018
23.05.2018
23.05.2018
22.05.2018
22.05.2018
21.05.2018
21.05.2018
19.05.2018


KeyboardInterrupt: 